### Analysis of NYC 311 records assigned to Department of Housing Preservation and Development (HPD)

This notebook uses the New York City Data Portal Python API to pull in all 311 data records (from January 1, 2010 to Oct 30, 2020 - present) that were assigned to the HPD for resolution. It conducts a tokenization analysis of the resolution description found in this dataset and generates tables that can be fed into Tableau to visualize. It is organized into the following sections:

- **Intro**: Configuration and helpful functions
- **Section 1**: Pulls in a sample of data from all departments to make sure that all complaints that should be assigned to HPD are in fact correctly labeled
- **Section 2**: Pulls in all data from 2010-present assigned to HPD and performs basic cleaning
- **Section 3**: Performs more sophisticated cleaning of the resolution_descriptions (ie. a few sentences describing the steps taken to resolve the complaint, and the outcome if completed)
- **Section 4**: Assigns text tags to the resolution_descriptions (ie. tokenize the resolution_descriptions into important phrases that are common to all of them). This enables me to analyze the sentiment of these descriptions and better understand the HPD resolution process and citizen's experience when using 311.
- **Section 5**: Performs logical checks on the data integrity and resolves problems (ex. ensure the timestamp that a complaint is closed falls after the timestamp of when it's open)
-  **Section 6**: Analyzes resolution_description text_tags and exports tables that can be loaded into Tableau to generate visualizations

The final visualizations that this dataset produces can be viewed here:
1. https://public.tableau.com/views/Atokenizedtextanalysisof6_2millionNYC311publichousingclaims/ExploratoryDashboard?:language=en&:display_count=y&:origin=viz_share_link
2. https://public.tableau.com/views/Atokenizedtextanalysisof6_2millionNYC311publichousingclaims/InsightsfromExploration?:language=en&:display_count=y&:origin=viz_share_link

## Intro
_Configuration and helpful functions_

In [288]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import geopandas as gpd

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.options.display.max_colwidth = 2000

In [4]:
def clean_311_dtypes(df_):
    '''
    Takes the 311 raw data as a pandas dataframe and returns a cleaned version of the dataframe
    '''
    #Format datatypes
    date_cols = ['created_date','closed_date', 'due_date', 'resolution_action_updated_date']
    for date_col in date_cols:
        df_[date_col] = pd.to_datetime(df_[date_col])

    #Make complaints all uppercase
    df_['complaint_type'] = df_['complaint_type'].str.upper()
    
    #Strip whitespace at beginning and end of all columns with text datatype
    df_text_object = df_.select_dtypes(include='object')
    for text_col in df_text_object.columns:
        df_[text_col] = df_[text_col].str.strip()
    
    return df_


## Section 1
### Load sample of all departments' data

_Pulls in a sample of data (50,000 records from 2019) from all departments to make sure that all complaints that should be assigned to HPD are in fact correctly labeled_

In [5]:
'''Use API to load data
- Socrata naturally has a limit of 1,000 records per query but got around this using below:
https://support.socrata.com/hc/en-us/articles/202949268-How-to-query-more-than-1000-rows-of-a-dataset
- Using an app token to avoid throttling
'''
#Limit to a given time period
app_token = ''#INSERT APP TOKEN HERE
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json?$where=created_date >'2019-01-01T00:00:00.000' AND created_date <'2019-12-31T00:00:00.000'&$limit=50000&$$app_token={}".format(app_token)
payload = {}
headers= {}
response = requests.request("GET", url, headers=headers, data = payload)
df_311 = pd.read_json(response.text)

#Clean data
df_311 = clean_311_dtypes(df_311)
print(df_311.shape[0])
display(df_311.head(2))

50000


,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,community_board,complaint_type,created_date,cross_street_1,cross_street_2,descriptor,due_date,facility_type,incident_address,incident_zip,intersection_street_1,intersection_street_2,landmark,latitude,location,location_type,longitude,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,vehicle_type,x_coordinate_state_plane,y_coordinate_state_plane
0,ADDRESS,HPD,Department of Housing Preservation and Development,3.019820e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-01-17 08:36:51,02 BROOKLYN,DOOR/WINDOW,2019-01-01 00:00:12,NaN,NaN,DOOR FRAME,NaT,N/A,435 GRAND AVENUE,11238,NaN,NaN,NaN,40.683022,NaN,RESIDENTIAL BUILDING,-73.961724,PHONE,BROOKLYN,Unspecified,2019-01-17 08:36:51,"The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists, please file a new complaint.",NaN,Closed,GRAND AVENUE,NaN,NaN,41450894,NaN,994866.0,188122.0
1,ADDRESS,NYPD,New York City Police Department,3.005570e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-01-01 02:22:46,06 BROOKLYN,NOISE - RESIDENTIAL,2019-01-01 00:00:29,KING STREET,SULLIVAN STREET,Loud Music/Party,2019-01-01 08:00:29,Precinct,135 RICHARDS STREET,11231,NaN,NaN,NaN,40.677200,NaN,Residential Building/House,-74.010351,MOBILE,BROOKLYN,Unspecified,2019-01-01 02:22:46,The Police Department responded to the complaint and a report was prepared.,NaN,Closed,RICHARDS STREET,NaN,NaN,41312454,NaN,981379.0,185999.0


In [6]:
#Look at allocation of responsibilities for each department
grouped_dep_claim = pd.DataFrame(df_311.groupby(['agency', 'complaint_type'])['unique_key'].count())
# display(grouped_dep_claim)

#Which complaints are sometimes sent to different departments
print("The following claims are sometimes sent to different departments:")
grouped_dep_claim['claim_sent_diff_dep'] = list(grouped_dep_claim.reset_index().duplicated(subset = ['complaint_type'], keep=False))
display(grouped_dep_claim[grouped_dep_claim['claim_sent_diff_dep']].sort_values('complaint_type'))


The following claims are sometimes sent to different departments:


,,unique_key,claim_sent_diff_dep
agency,complaint_type,,
DEP,ASBESTOS,25,True
DOHMH,ASBESTOS,7,True
DOB,ELEVATOR,293,True
HPD,ELEVATOR,23,True
DSNY,GRAFFITI,593,True
NYPD,GRAFFITI,3,True
DOB,PLUMBING,113,True
HPD,PLUMBING,988,True
DCAS,QUESTION,22,True


#### Why is some elevator and plumbing data sent to DOB instead of HPD? Does the location type explain why these things are sent to non-HPD departments?


In [7]:
#Organize by location type
grouped_agency_location = pd.DataFrame(df_311.groupby(['agency', 'location_type'])['unique_key'].count())
display(grouped_agency_location.loc[['HPD', 'DOB']])

,,unique_key
agency,location_type,
HPD,RESIDENTIAL BUILDING,12742


##### Interestingly, DOB does not show up in the above table which suggests the location_type is NaN for DOB claims. Can see that it is only ever 'Residential' for HPD claims. Suggests that they actually are different and are correctly routed.

#### Are the complaint types in HPD as expected?

In [8]:
grouped_dep_claim = pd.DataFrame(df_311.groupby(['agency', 'complaint_type', 'location_type'])['created_date'].count())
grouped_dep_claim.reset_index(inplace = True)
grouped_dep_claim[grouped_dep_claim['agency'].isin(['HPD'])]

,agency,complaint_type,location_type,created_date
187,HPD,APPLIANCE,RESIDENTIAL BUILDING,285
188,HPD,DOOR/WINDOW,RESIDENTIAL BUILDING,667
189,HPD,ELECTRIC,RESIDENTIAL BUILDING,421
190,HPD,ELEVATOR,RESIDENTIAL BUILDING,23
191,HPD,FLOORING/STAIRS,RESIDENTIAL BUILDING,444
192,HPD,GENERAL,RESIDENTIAL BUILDING,455
193,HPD,HEAT/HOT WATER,RESIDENTIAL BUILDING,5744
194,HPD,OUTSIDE BUILDING,RESIDENTIAL BUILDING,18
195,HPD,PAINT/PLASTER,RESIDENTIAL BUILDING,1046
196,HPD,PLUMBING,RESIDENTIAL BUILDING,988


##### Seems that complaints are correctly assigned to HPD. Going to move on to only pulling those complaints, but for all years.

## Section 2
### Load all HPD data
Pulls in all data from 2010-present assigned to HPD and performs basic cleaning

In [9]:
#Need to iterate through 50,000 blocks using the API to load in all the data for HPD

#Iteration parameters
app_token = ''#INSERT APP TOKEN HERE
current_date='2020-09-30T00:00:00.000'
number_rows = 23914593
chunk = 50000
max_iter = round(number_rows/chunk)

#Iterate using API
chunks=[]
for i in range(max_iter):
    offset = i*chunk
    url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json?agency=HPD&$where=created_date<'{}'&$limit={}&$offset={}&$$app_token={}".format(current_date, chunk, offset, app_token)
    print('Iter: {}'.format(i))
    payload = {}
    headers= {}

    response = requests.request("GET", url, headers=headers, data = payload)
    if (response.status_code == 200) & ('[]' not in response.text):
        df_311_chunk = pd.read_json(response.text)
        chunks.append(df_311_chunk)
    else:
        break

Iter: 0
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
Iter: 20
Iter: 21
Iter: 22
Iter: 23
Iter: 24
Iter: 25
Iter: 26
Iter: 27
Iter: 28
Iter: 29
Iter: 30
Iter: 31
Iter: 32
Iter: 33
Iter: 34
Iter: 35
Iter: 36
Iter: 37
Iter: 38
Iter: 39
Iter: 40
Iter: 41
Iter: 42
Iter: 43
Iter: 44
Iter: 45
Iter: 46
Iter: 47
Iter: 48
Iter: 49
Iter: 50
Iter: 51
Iter: 52
Iter: 53
Iter: 54
Iter: 55
Iter: 56
Iter: 57
Iter: 58
Iter: 59
Iter: 60
Iter: 61
Iter: 62
Iter: 63
Iter: 64
Iter: 65
Iter: 66
Iter: 67
Iter: 68
Iter: 69
Iter: 70
Iter: 71
Iter: 72
Iter: 73
Iter: 74
Iter: 75
Iter: 76
Iter: 77
Iter: 78
Iter: 79
Iter: 80
Iter: 81
Iter: 82
Iter: 83
Iter: 84
Iter: 85
Iter: 86
Iter: 87
Iter: 88
Iter: 89
Iter: 90
Iter: 91
Iter: 92
Iter: 93
Iter: 94
Iter: 95
Iter: 96
Iter: 97
Iter: 98
Iter: 99
Iter: 100
Iter: 101
Iter: 102
Iter: 103
Iter: 104
Iter: 105
Iter: 106
Iter: 107
Iter: 108
Iter: 109
Iter: 110


In [10]:
#Concat chunks into one dataset
df_311 = pd.concat(chunks, sort = False)

#Clean data
df_311 = clean_311_dtypes(df_311)
display(df_311.head(2))

#Check for duplicates in case the offsets were inclusive of the domain and are overlapping
df_311_no_duplicates = df_311.drop_duplicates(subset = 'unique_key')

if df_311_no_duplicates.shape[0] == df_311.shape[0]:
    df_311 = df_311_no_duplicates.copy()
    del df_311_no_duplicates

print('Number of records assigned to the HPD: {}'.format(df_311.shape[0]))

,address_type,agency,agency_name,bbl,borough,city,closed_date,community_board,complaint_type,created_date,cross_street_1,cross_street_2,descriptor,incident_address,incident_zip,intersection_street_1,intersection_street_2,landmark,latitude,location,location_type,longitude,open_data_channel_type,park_borough,park_facility_name,resolution_action_updated_date,resolution_description,status,street_name,unique_key,x_coordinate_state_plane,y_coordinate_state_plane,due_date,facility_type
0,ADDRESS,HPD,Department of Housing Preservation and Development,3.074330e+09,BROOKLYN,BROOKLYN,NaT,15 BROOKLYN,WATER LEAK,2020-09-29 17:51:04,NaN,NaN,SLOW LEAK,1213 AVENUE Z,11235.0,NaN,NaN,NaN,40.587653,NaN,RESIDENTIAL BUILDING,-73.957289,PHONE,BROOKLYN,Unspecified,2020-09-29,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,Open,AVENUE Z,47732947,996113.0,153377.0,NaT,NaN
1,ADDRESS,HPD,Department of Housing Preservation and Development,3.013160e+09,BROOKLYN,BROOKLYN,NaT,09 BROOKLYN,UNSANITARY CONDITION,2020-09-29 00:22:57,NaN,NaN,PESTS,348 EMPIRE BOULEVARD,11225.0,NaN,NaN,NaN,40.663922,NaN,RESIDENTIAL BUILDING,-73.950589,ONLINE,BROOKLYN,Unspecified,2020-09-29,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,Open,EMPIRE BOULEVARD,47732845,997958.0,181165.0,NaT,NaN


Number of records assigned to the HPD: 6207126


In [11]:
#Export data to csv to avoid having to run long chunks query again
df_311.to_csv("cleaned_data/HPD_311_all.csv")

## Section 3
### Clean resolution_description text
One of the most helpful variables in the 311 dataset is a resolution_description: a few sentences describing the steps taken to resolve the complaint, and the outcome if available. This field provides rich information about each claim, but is difficult to use because of some inconsistencies in how it's written, and issues with how it's saved. In the following I fix these issues:

- Fixed text of truncated entries (There seems to be an issue with how they were saved and many records are randomly truncated midway through sentences. To the human eye they are clearly duplicates of other entries which are written in their entirety, but because of the truncation they appear distinct to a computer. I've made every entry the full, untruncated form.)
- Make period/spacing usage uniform
- Remove commas

In [12]:
#Export all possible resolution_descriptions to manually fix truncation issue
pd.DataFrame({'unique_resolution_description':df_311['resolution_description'].unique()}).to_csv("cleaning_resolution_text/unique_resolution_type.csv")

##### Truncation issue explanation -
From the above, it appears that there are 343 distinct possibilities of what can be entered into the resolution_description, but most of these are just truncated versions of each other. I've manually gone through and created a dictionary that relates each truncated form to the non-truncated form. This analysis reveals that there are actually only 40 distinct possibility of what is saved in the resolution_description. Loading this dictionary below and finishing the cleaning:

In [13]:
#Load in dict to fix truncation issue
resolution_dict = pd.read_csv("cleaning_resolution_text/manual_resolution_desc_cleaning.csv")

#Remove all commas
resolution_dict['cleaned_resolution_description'] = resolution_dict['cleaned_resolution_description'].str.replace(',','')

#Make sure there are only single spaces between words
resolution_dict['cleaned_resolution_description'] = resolution_dict['cleaned_resolution_description'].str.replace(r" +", ' ')

#Put a space after every period
resolution_dict['cleaned_resolution_description'] = resolution_dict['cleaned_resolution_description'].str.replace(r"(?<=[.,])(?=[^\s])", ' ')

#Make an uppercase version
resolution_dict['cleaned_resolution_description_upper'] = resolution_dict['cleaned_resolution_description'].str.upper()

#Merge in the cleaned_resolution_description to the full 311 dataframe
df_311 = df_311.merge(resolution_dict[['resolution_description', 'cleaned_resolution_description', 'cleaned_resolution_description_upper']], on = 'resolution_description', how = 'left')

## Section 4
### Generate text tags
*Assigns text tags to the resolution_descriptions (ie. tokenize the resolution_descriptions into important phrases that are common to all of them). This enables me to analyze the sentiment of these descriptions and better understand the HPD resolution process and citizen's experience when using 311.*

A manual analysis of the 40 cleaned_resolution_descriptions present (ie. untruncated versions), reveals that there are only 45 distinct pieces of information contained in them. I've extracted the phrases that correspond to these pieces of information and saved them each as a text_tag in the text_tag_dict.csv. I then append a binary flag column for each text_tag to the 311 dataset and mark for each record's resolution_description which text_tags are present. 

While extracting the text_tags, I also noticed that they can be categorized into four types of information:
1. process: The steps that the HPD has taken to learn more about the issue, and the barriers they faced while doing it. 
2. outcome: The final decision made concerning the complaint and/or the status of the complaint.
3. next_steps: Further steps that the HPD might take or that the user can take to learn more about the outcome and/or pursue further action.
4. record-type: Attributes of the complaint such as whether it's a building-wide complaint or particular to a certain apartment.

Creating and organizing these text_tags enables me to conduct a sentiment analysis in Section 6 to better understand the resolution process and user experience.


In [24]:
#Export set of unique cleaned resolution descriptions to analyze and generate tags
pd.DataFrame({'unique_cleaned_resolution_description':resolution_dict['cleaned_resolution_description'].unique()}).to_csv("cleaning_resolution_text/distinct_cleaned_resolution_descriptions.csv")

In [25]:
#Load in dictionary of manually generated text_tags
text_tag_dict = pd.read_csv("cleaning_resolution_text/text_tag_dict.csv")
print('The text tags I generated, grouped by category are:')
display(pd.DataFrame(text_tag_dict.groupby(['tag_type', 'text_tag']).count()))

The text tags I generated, grouped by category are:


Empty DataFrame
Columns: []
Index: [(next_steps, 311 CAN SUBMIT A NEW REQUEST TO BE SENT BY DIRECT MAIL), (next_steps, CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION), (next_steps, IF IT HAS NOT BEEN RECEIVED PLEASE REQUEST IT AGAIN), (next_steps, IF IT IS NOT RECEIVED PLEASE CALL 311 AND SUBMIT A NEW REQUEST), (next_steps, IF THE CONDITION STILL EXISTS PLEASE FILE A NEW COMPLAINT), (next_steps, IF THE CONDITIONS STILL EXIST AND AN INSPECTION IS REQUIRED PLEASE CONTACT THE BOROUGH OFFICE WITH YOUR COMPLAINT NUMBER), (next_steps, INFORMATION ABOUT INSPECTION RESULTS CAN BE FOUND THROUGH HPD'S WEBSITE), (next_steps, INFORMATION ABOUT SPECIFIC VIOLATIONS IS AVAILABLE AT WWW. NYC. GOV/HPD), (next_steps, INSPECTION TO TEST THE PAINT FOR LEAD), (next_steps, MAY ATTEMPT TO CONTACT YOU), (next_steps, MORE INFORMATION CAN BE FOUND THROUGH HPD'S WEBSITE), (next_steps, PLEASE NOTE YOUR SERVICE REQUEST NUMBER FOR FUTURE REFERENCE), (next_steps, SCHEDULE A FOLLOW-UP INSPECTION), (next_steps, TENANT AND THE PROPERTY OWNER WILL RECEIVE A NOTICE IN THE MAIL), (next_steps, WILL ATTEMPT TO CONTACT YOU), (outcome, ALSO IDENTIFIED POTENTIAL LEAD), (outcome, CLOSED THIS COMPLAINT ADMINISTRATIVELY), (outcome, COMPLAINT CONDITIONS ARE STILL OPEN), (outcome, COMPLAINT HAS BEEN CLOSED), (outcome, COMPLAINT WAS ADDRESSED), (outcome, CONDITIONS WERE CORRECTED), (outcome, DID NOT HAVE ENOUGH TIME TO INSPECT), (outcome, HEAT OR HOT WATER WAS INADEQUATE), (outcome, HEAT WAS NOT REQUIRED), (outcome, IDENTIFIED POTENTIAL LEAD), (outcome, LITERATURE HAS BEEN MAILED), (outcome, LITERATURE WAS EMAILED BUT WAS NOT RECEIVED), (outcome, LITERATURE WILL BE EMAILED WITHIN 24 HOURS), (outcome, NO VIOLATIONS WERE ISSUED), (outcome, NOT ABLE TO GAIN ACCESS), (outcome, ORIGINAL COMPLAINT IS STILL OPEN), (outcome, RESTORED), (outcome, SECTION 8 FAILURE WAS ISSUED), (outcome, UNABLE TO ACCESS), (outcome, VIOLATION WAS ISSUED), (outcome, VIOLATIONS WERE ISSUED), (outcome, VIOLATIONS WERE PREVIOUSLY ISSUED), (process, ADVISED BY A TENANT IN THE BUILDING), (process, AT THE TIME OF THE INSPECTION), (process, BY INSPECTING ANOTHER APARTMENT), (process, CONTACTED A TENANT), (process, CONTACTED AN OCCUPANT), (process, INSPECTED THE FOLLOWING CONDITIONS), (process, MAY CONDUCT AN INSPECTION), (record_type, BUILDING-WIDE CONDITION), (record_type, COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT), (record_type, HEAT OR HOT WATER)]

In [16]:
#DELETE------------------XXXXXX
unique_cleaned_resolutions = pd.DataFrame({'unique_cleaned_resolution_description_upper':resolution_dict['cleaned_resolution_description_upper'].unique()})

#Append binary columns flagging each in each record
text_tags_not_found = []
for index, row in text_tag_dict.iterrows():
    tag_to_find = row['text_tag']
    print(tag_to_find)
    unique_cleaned_resolutions[tag_to_find] = 0
    indices_update = unique_cleaned_resolutions.index[unique_cleaned_resolutions['unique_cleaned_resolution_description_upper'].str.contains(tag_to_find)]
    if indices_update.empty:
        text_tags_not_found.append(tag_to_find)
    else:
        unique_cleaned_resolutions.loc[indices_update,tag_to_find] = 1
        
if not text_tags_not_found:
    print('The following text tags were not found in the dataset: {} Check their spelling for errors.'.format(text_tags_not_found))

BUILDING-WIDE CONDITION
HEAT OR HOT WATER
COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT
CONDITIONS WERE CORRECTED
COMPLAINT WAS ADDRESSED
COMPLAINT HAS BEEN CLOSED
NO VIOLATIONS WERE ISSUED
VIOLATIONS WERE PREVIOUSLY ISSUED
VIOLATION WAS ISSUED
IDENTIFIED POTENTIAL LEAD
ALSO IDENTIFIED POTENTIAL LEAD
CLOSED THIS COMPLAINT ADMINISTRATIVELY
SECTION 8 FAILURE WAS ISSUED
RESTORED
HEAT WAS NOT REQUIRED
DID NOT HAVE ENOUGH TIME TO INSPECT
HEAT OR HOT WATER WAS INADEQUATE
COMPLAINT CONDITIONS ARE STILL OPEN
ORIGINAL COMPLAINT IS STILL OPEN
LITERATURE HAS BEEN MAILED
LITERATURE WAS EMAILED BUT WAS NOT RECEIVED
LITERATURE WILL BE EMAILED WITHIN 24 HOURS
CONTACTED A TENANT
CONTACTED AN OCCUPANT
INSPECTED THE FOLLOWING CONDITIONS
ADVISED BY A TENANT IN THE BUILDING
NOT ABLE TO GAIN ACCESS
UNABLE TO ACCESS
BY INSPECTING ANOTHER APARTMENT
MAY ATTEMPT TO CONTACT YOU
WILL ATTEMPT TO CONTACT YOU
INSPECTION TO TEST THE PAINT FOR LEAD
CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION
IF THE CONDITION STIL

In [17]:
unique_cleaned_resolutions.to_csv("cleaning_resolution_text/unique_cleaned_text_resolutions_with_tags.csv")

In [18]:
#Change the records that are missing a resolution description from NaN to empty string to avoid indexing error when looking for substrings
df_311.loc[df_311['cleaned_resolution_description_upper'].isna(), 'cleaned_resolution_description_upper'] = ''

#Append binary columns flagging each in each record
text_tags_not_found = []
for index, row in text_tag_dict.iterrows():
    tag_to_find = row['text_tag']
    print(tag_to_find)
    df_311[tag_to_find] = 0
    indices_update = df_311.index[df_311['cleaned_resolution_description_upper'].str.contains(tag_to_find)]
    if indices_update.empty:
        text_tags_not_found.append(tag_to_find)
    else:
        df_311.loc[indices_update,tag_to_find] = 1
        
if not text_tags_not_found:
    print('The following text tags were not found in the dataset: {} Check their spelling for errors.'.format(text_tags_not_found))

BUILDING-WIDE CONDITION
HEAT OR HOT WATER
COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT
CONDITIONS WERE CORRECTED
COMPLAINT WAS ADDRESSED
COMPLAINT HAS BEEN CLOSED
NO VIOLATIONS WERE ISSUED
VIOLATIONS WERE PREVIOUSLY ISSUED
VIOLATION WAS ISSUED
IDENTIFIED POTENTIAL LEAD
ALSO IDENTIFIED POTENTIAL LEAD
CLOSED THIS COMPLAINT ADMINISTRATIVELY
SECTION 8 FAILURE WAS ISSUED
RESTORED
HEAT WAS NOT REQUIRED
DID NOT HAVE ENOUGH TIME TO INSPECT
HEAT OR HOT WATER WAS INADEQUATE
COMPLAINT CONDITIONS ARE STILL OPEN
ORIGINAL COMPLAINT IS STILL OPEN
LITERATURE HAS BEEN MAILED
LITERATURE WAS EMAILED BUT WAS NOT RECEIVED
LITERATURE WILL BE EMAILED WITHIN 24 HOURS
CONTACTED A TENANT
CONTACTED AN OCCUPANT
INSPECTED THE FOLLOWING CONDITIONS
ADVISED BY A TENANT IN THE BUILDING
NOT ABLE TO GAIN ACCESS
UNABLE TO ACCESS
BY INSPECTING ANOTHER APARTMENT
MAY ATTEMPT TO CONTACT YOU
WILL ATTEMPT TO CONTACT YOU
INSPECTION TO TEST THE PAINT FOR LEAD
CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION
IF THE CONDITION STIL

In [19]:
#Check if there are any records with resolution descriptions that didn't qualify for any text tags (there shouldn't be)
all_tags = list(text_tag_dict['text_tag'].values)
if not df_311[(df_311[all_tags].sum(axis = 1) == 0) & (~df_311['resolution_description'].isna())].empty:
    print("There are untagged records with valid resolution descriptions. Add tags to cover these instances.")

In [20]:
#Fix text tags that are subsets of each other
no_violations = df_311[df_311['NO VIOLATIONS WERE ISSUED'] == 1].index
df_311.loc[no_violations, 'VIOLATIONS WERE ISSUED'] = 0

also_identified_lead = df_311[df_311['ALSO IDENTIFIED POTENTIAL LEAD'] == 1].index
df_311.loc[also_identified_lead, 'IDENTIFIED POTENTIAL LEAD'] = 0

In [75]:
#Use hierarchy of outcomes (manually analyzed from text tags) to create an ultimate outcome column
outcome_order = pd.read_csv("cleaning_resolution_text/hierarchical_outcome_text_tags.csv")

df_311['ultimate_outcome'] = ''
counter = 0
for index, outcome_row in outcome_order.iterrows():
    text_tag = outcome_row['text_tag']
    print(text_tag)
    if counter == 0:
        update_indices = df_311[df_311[text_tag] == 1].index
    else:
        update_indices = df_311[(df_311[text_tag] == 1) & (df_311['ultimate_outcome'] == '')].index
    df_311.loc[update_indices, 'ultimate_outcome'] = text_tag
    counter +=1
    
#A couple resolution descriptions don't specify an outcome or whether the claim is closed
#Label as 'COMPLAINT CONDITIONS ARE STILL OPEN' if status == open and 'OUTCOME UNKNOWN' if status is closed
unknown_open = df_311[(df_311['ultimate_outcome'] == '') & (df_311['status'] == 'Open')].index
unknown_closed = df_311[(df_311['ultimate_outcome'] == '') & (df_311['status'] != 'Open')].index
df_311.loc[unknown_open, 'ultimate_outcome'] = 'COMPLAINT CONDITIONS ARE STILL OPEN'
df_311.loc[unknown_closed, 'ultimate_outcome'] = 'OUTCOME UNKNOWN CLOSED'
df_311.loc[unknown_open, 'COMPLAINT CONDITIONS ARE STILL OPEN'] = 1
df_311['OUTCOME UNKNOWN CLOSED'] = 0
df_311.loc[unknown_closed, 'OUTCOME UNKNOWN CLOSED']=1

CONDITIONS WERE CORRECTED
COMPLAINT WAS ADDRESSED
NO VIOLATIONS WERE ISSUED
VIOLATION WAS ISSUED
VIOLATIONS WERE ISSUED
VIOLATIONS WERE PREVIOUSLY ISSUED
RESTORED
SECTION 8 FAILURE WAS ISSUED
NOT ABLE TO GAIN ACCESS
UNABLE TO ACCESS
COMPLAINT HAS BEEN CLOSED
COMPLAINT CONDITIONS ARE STILL OPEN
ORIGINAL COMPLAINT IS STILL OPEN
IDENTIFIED POTENTIAL LEAD
ALSO IDENTIFIED POTENTIAL LEAD
CLOSED THIS COMPLAINT ADMINISTRATIVELY
HEAT WAS NOT REQUIRED
DID NOT HAVE ENOUGH TIME TO INSPECT
HEAT OR HOT WATER WAS INADEQUATE
LITERATURE HAS BEEN MAILED
LITERATURE WAS EMAILED BUT WAS NOT RECEIVED
LITERATURE WILL BE EMAILED WITHIN 24 HOURS


In [76]:
#Add process tags for outcomes that imply process
no_access = df_311[(df_311['UNABLE TO ACCESS'] == 1) | (df_311['NOT ABLE TO GAIN ACCESS'] == 1) | (df_311['DID NOT HAVE ENOUGH TIME TO INSPECT']==1)].index
df_311['ATTEMPTED AND FAILED TO INVESTIGATE ORIGINAL PROPERTY'] = 0
df_311.loc[no_access, 'ATTEMPTED AND FAILED TO INVESTIGATE ORIGINAL PROPERTY'] = 1

literature_cols = [column for column in df_311.columns if 'LITERATURE' in column]
literature_records = df_311[df_311[literature_cols].sum(axis =1) != 0].index
df_311['LITERATURE PREPARATION'] = 0
df_311.loc[literature_records, 'LITERATURE PREPARATION'] = 1

In [95]:
#Use hierarchical process (manually analyzed) to create an ultimate process column
process_order = pd.read_csv("cleaning_resolution_text/hierarchical_process_text_tags.csv")

df_311['ultimate_process'] = ''
counter = 0
for index, process_row in process_order.iterrows():
    text_tag = process_row['text_tag']
    print(text_tag)
    if counter == 0:
        update_indices = df_311[df_311[text_tag] == 1].index
    else:
        update_indices = df_311[(df_311[text_tag] == 1) & (df_311['ultimate_process'] == '')].index

    df_311.loc[update_indices, 'ultimate_process'] = text_tag
    counter +=1
    
#A few resolution descriptions don't specify process - mark these
process_unknown = df_311[df_311['ultimate_process'] == ''].index
df_311['RESOLUTION PROCESS UNKNOWN'] = 0
df_311.loc[process_unknown, 'RESOLUTION PROCESS UNKNOWN']=1
df_311.loc[process_unknown, 'ultimate_process'] = 'RESOLUTION PROCESS UNKNOWN'

BY INSPECTING ANOTHER APARTMENT
CONTACTED A TENANT
CONTACTED AN OCCUPANT
INSPECTED THE FOLLOWING CONDITIONS
ADVISED BY A TENANT IN THE BUILDING
AT THE TIME OF THE INSPECTION
ATTEMPTED AND FAILED TO INVESTIGATE ORIGINAL PROPERTY
LITERATURE PREPARATION
CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION


In [96]:
# #Append newly created text_tags to the data_dict
new_rows = [{'text_tag':'OUTCOME UNKNOWN CLOSED', 'tag_type':'outcome'},{'text_tag':'ATTEMPTED AND FAILED TO INVESTIGATE ORIGINAL PROPERTY', 'tag_type':'process'}, {'text_tag':'LITERATURE PREPARATION', 'tag_type':'process'}, {'text_tag':'RESOLUTION PROCESS UNKNOWN', 'tag_type':'process'}] 

for new_row in new_rows:
    text_tag_dict = text_tag_dict.append(new_row, ignore_index=True)

In [97]:
#Check ultimate outcomes
display(pd.DataFrame(df_311.groupby(['ultimate_outcome'])['unique_key'].count()))
display(pd.DataFrame(df_311.groupby(['ultimate_process'])['unique_key'].count()))

,unique_key
ultimate_outcome,
CLOSED THIS COMPLAINT ADMINISTRATIVELY,24930
COMPLAINT CONDITIONS ARE STILL OPEN,143288
COMPLAINT WAS ADDRESSED,12607
CONDITIONS WERE CORRECTED,696653
DID NOT HAVE ENOUGH TIME TO INSPECT,13
IDENTIFIED POTENTIAL LEAD,38814
LITERATURE HAS BEEN MAILED,33113
LITERATURE WILL BE EMAILED WITHIN 24 HOURS,18058
NO VIOLATIONS WERE ISSUED,1779300


,unique_key
ultimate_process,
ADVISED BY A TENANT IN THE BUILDING,433882
AT THE TIME OF THE INSPECTION,3824
ATTEMPTED AND FAILED TO INVESTIGATE ORIGINAL PROPERTY,1251961
BY INSPECTING ANOTHER APARTMENT,5236
CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION,90990
CONTACTED A TENANT,696653
CONTACTED AN OCCUPANT,12607
INSPECTED THE FOLLOWING CONDITIONS,2948062
LITERATURE PREPARATION,51171


## Section 5
### Logical checks on data integrity

*Performs logical checks on the data integrity and resolves problems (ex. ensure the timestamp that a complaint is closed falls after the timestamp of when it's open)*

#### Are all created_dates after the the opened timestamps?

In [98]:
closed_before_opened = df_311['closed_date']<df_311['created_date']
if any(closed_before_opened):
    print('{} records have closed dates that fall before created dates. Dropping these records.'.format(sum(closed_before_opened)))
    df_311_cleaned = df_311[~closed_before_opened]

2 records have closed dates that fall before created dates. Dropping these records.


#### Are all due_dates after the the opened timestamps?

In [99]:
due_before_opened = df_311_cleaned['due_date']<df_311_cleaned['created_date']
if any(due_before_opened):
    print('{} records have due dates that fall before created dates. Dropping these records.'.format(sum(closed_before_opened)))
    df_311_cleaned = df_311_cleaned[~due_before_opened]

#### Are any of the resolution_action_updated_dates after the closed_dates? 

In [100]:
updated_after_closed = df_311_cleaned['resolution_action_updated_date']>df_311_cleaned['closed_date']
if any(updated_after_closed):
    print('{} records have updated dates that fall after closed dates. Analyzing below:'.format(sum(updated_after_closed)))
    updated_after_closed_df = df_311_cleaned[updated_after_closed]
    
    #Calculate amount of time update falls after closed time in seconds - many are just 1 second
    updated_after_closed_df['updated_closed_delta'] = (updated_after_closed_df['resolution_action_updated_date'] - updated_after_closed_df['closed_date']).astype('timedelta64[s]')
    print("{} of these records' 'update' timestamps are only 1 second after the 'closed' timestamps (ie. seem to just be due to a system lag)".format(sum(updated_after_closed_df['updated_closed_delta'] == 1)))
    
    print('Examining the ones that are updated more than a day after the closed date:')
    more_than_day_delta = updated_after_closed_df[updated_after_closed_df['updated_closed_delta']>(60*60*24)]
#     display(more_than_day_delta.head(2))
    display(pd.DataFrame(more_than_day_delta.groupby(['cleaned_resolution_description', 'status'])['unique_key'].count()))

    print('There are {} records where the updated date is more than a day after the closed date. Seems likely these are an error. Dropping these and keeping the rest.'.format(more_than_day_delta.shape[0]))
    
    #Drop the ones updated more than a day after the closed date?
    unique_keys_drop = more_than_day_delta['unique_key']
    df_311_cleaned = df_311_cleaned[~df_311_cleaned['unique_key'].isin(unique_keys_drop)]

1343 records have updated dates that fall after closed dates. Analyzing below:
933 of these records' 'update' timestamps are only 1 second after the 'closed' timestamps (ie. seem to just be due to a system lag)
Examining the ones that are updated more than a day after the closed date:


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


unique_key
cleaned_resolution_description                                                                                                                                                                                                                                                                                                                                            status            
More than one complaint was received for this building-wide condition. This complaint status is for the initial complaint. The Department of Housing Preservation and Development contacted a tenant in the building and verified that the following conditions were corrected. The complaint has been closed. If the condition still exists please file a new complaint. Closed          29
The Department of Housing Preservation and Development conducted an inspection for the following conditions and identified potential lead-based paint conditions. HPD will attempt to contact you to schedule a follow-up inspection to test the paint for lead.                                                                                                          Closed           2
The Department of Housing Preservation and Development contacted a tenant in the building and verified that the following conditions were corrected. The complaint has been closed. If the condition still exists please file a new complaint.                                                                                                                            Closed           2
The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.                                                                                                                                                                                                                      Closed         136
The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www. nyc. gov/hpd.                                                                                                                                                                               Closed          16
                                                                                                                                                                                                                                                                                                                                                                          Open             1
The Department of Housing Preservation and Development inspected the following conditions. Violations were previously issued for these conditions. Information about specific violations is available at www. nyc. gov/hpd.                                                                                                                                               Closed          12
The Department of Housing Preservation and Development responded to a complaint of no heat or hot water and was advised by a tenant in the building that heat and hot water had been restored. If the condition still exists please file a new complaint.                                                                                                                 Closed           2
The Department of Housing Preservation and Development was not able to gain access to inspect the following conditions. The complaint has been closed. If the condition still exists please file a new complaint.                                                                                                                                                         Closed          12
The Department of Housing Preservation and Development was not able to gain access to your apartment or others in the building to inspect for a lack of heat or hot water. However 

There are 230 records where the updated date is more than a day after the closed date. Seems likely these are an error. Dropping these and keeping the rest.


#### Are there any instances where the created_date is exactly the same as the closed_date timestamp?


In [101]:
created_closed_same_time = df_311_cleaned['created_date']==df_311_cleaned['closed_date']
if any(created_closed_same_time):
    print('{} records are closed at the exact same time they are opened. Looking into resolution data:'.format(sum(created_closed_same_time)))
    created_closed_same_time_df = df_311_cleaned[created_closed_same_time]
    display(pd.DataFrame(created_closed_same_time_df.groupby(['cleaned_resolution_description', 'status'])['unique_key'].count()))
    
#Keep the ones that are building wide conditions and drop the rest
created_closed_drop = df_311_cleaned[(created_closed_same_time) & (df_311_cleaned['BUILDING-WIDE CONDITION'] == 0)].index
print('Dropping the {} records that were closed at the exact time they were opened that do not belong to a duplicate complaint of a building-wide condition.'.format(len(list(created_closed_drop))))
df_311_cleaned.drop(created_closed_drop, inplace = True)

57549 records are closed at the exact same time they are opened. Looking into resolution data:


,,unique_key
cleaned_resolution_description,status,
More than one complaint was received for this building-wide condition. This complaint status is for the initial complaint. The Department of Housing Preservation and Development contacted a tenant in the building and verified that the following conditions were corrected. The complaint has been closed. If the condition still exists please file a new complaint.,Closed,34174
The Department of Housing Preservation and Development conducted an inspection for the following conditions and identified potential lead-based paint conditions. HPD will attempt to contact you to schedule a follow-up inspection to test the paint for lead.,Closed,5
The Department of Housing Preservation and Development conducted or attempted to conduct an inspection. More information about inspection results can be found through HPD's website at www. nyc. gov/hpd by using HPDONLINE (enter your address on the home page) and entering your SR number under the complaint status option.,Closed,8421
The Department of Housing Preservation and Development contacted a tenant in the building and verified that the following conditions were corrected. The complaint has been closed. If the condition still exists please file a new complaint.,Closed,9625
The Department of Housing Preservation and Development contacted an occupant of the apartment or building and verified that the complaint was addressed. The complaint has been closed.,Closed,312
The Department of Housing Preservation and Development has closed this complaint administratively. More information can be found through HPD's website at www. nyc. gov/hpd by using HPDONLINE (enter your address on the home page) and entering your SR number under the complaint status option.,Closed,390
The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.,Closed,432
The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www. nyc. gov/hpd.,Closed,314
The Department of Housing Preservation and Development inspected the following conditions. Violations were previously issued for these conditions. Information about specific violations is available at www. nyc. gov/hpd.,Closed,44


Dropping the 23030 records that were closed at the exact time they were opened that do not belong to a duplicate complaint of a building-wide condition.


Rationale for dropping:
- Many of these fall into the catgories of 'duplicate, building-wide complaints'. It could make sense to close these at the time of opening if the agency just wants to avoid having duplicated complaints open. I've kept these.
- But others describe processes that would have taken time to execute (ex. inspection) and it doesn't seem possible that they could be closed at the time of opening. Handled these as errors by dropping them.
- How can a complaint be labeled as 'complaint conditions are still open' but have a closed_date at all? - look at these closer in the next section

#### Is the closed date 'NaN' for the service requests that are not closed?

In [102]:
#What are the options for the status of the service request? (helpful for understanding below)
print("The status options are: {}".format(set(df_311_cleaned['status'].unique())))

The status options are: {'Open', 'In Progress', 'Closed', 'Assigned'}


In [103]:
status_open_df = df_311_cleaned[df_311_cleaned['status'] != 'Closed']
open_with_closed_date_df = status_open_df[~status_open_df['closed_date'].isna()]
if not open_with_closed_date_df.empty:
    print("There are {} records with an 'open' status that have non_null values for closed_date. Analyzing below:".format(open_with_closed_date_df.shape[0]))
    display(pd.DataFrame(open_with_closed_date_df.groupby(['cleaned_resolution_description'])['unique_key'].count()))


There are 7122 records with an 'open' status that have non_null values for closed_date. Analyzing below:


,unique_key
cleaned_resolution_description,
More than one complaint was received for this building-wide condition. This complaint status is for the initial complaint. The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,61
The Department of Housing Preservation and Development conducted an inspection for the following conditions and identified potential lead-based paint conditions. HPD will attempt to contact you to schedule a follow-up inspection to test the paint for lead.,40
The Department of Housing Preservation and Development inspected the following conditions. No violations were issued. The complaint has been closed.,23
The Department of Housing Preservation and Development inspected the following conditions. Violations were issued. Information about specific violations is available at www. nyc. gov/hpd.,35
The Department of Housing Preservation and Development inspected the following conditions. Violations were previously issued for these conditions. Information about specific violations is available at www. nyc. gov/hpd.,2
The Department of Housing Preservation and Development responded to a complaint of no heat or hot water and was advised by a tenant in the building that heat and hot water had been restored. If the condition still exists please file a new complaint.,2
The Department of Housing Preservation and Development was not able to gain access to inspect the conditions. If the conditions still exist and an inspection is required please contact the borough office with your complaint number at 212 234 - 7485 (MANHATTAN).,637
The Department of Housing Preservation and Development was not able to gain access to inspect the conditions. If the conditions still exist and an inspection is required please contact the borough office with your complaint number at 718 286 - 0823 (QUEENS).,667
The Department of Housing Preservation and Development was not able to gain access to inspect the conditions. If the conditions still exist and an inspection is required please contact the borough office with your complaint number at 718 579-6777 (BRONX).,1510


- For some of these the resolution_description specifies if it's open or closed and conflicts with the closed_date nullity and the status listed. Not sure whether to trust the 'status', 'closed_date', or resolution_description to determine if it's open or closed. Using the text tags to more accurately identify these below.

In [104]:
#Comparing resolution description to status
closed_tags = ['COMPLAINT HAS BEEN CLOSED', 'CLOSED THIS COMPLAINT ADMINISTRATIVELY']
open_tags = ['COMPLAINT CONDITIONS ARE STILL OPEN']

resolution_closed_status_not = df_311_cleaned[(df_311_cleaned[closed_tags].sum(axis = 1) >= 1) & (df_311_cleaned['BUILDING-WIDE CONDITION'] != 1) & (df_311_cleaned['status'] != 'Closed')].index
print("There are {} records where the resolution description claims the complaint is closed but the status claims it's open. Dropping these.".format(len(list(resolution_closed_status_not))))
resolution_open_status_not = df_311_cleaned[(df_311_cleaned[open_tags[0]] == 1) & (df_311_cleaned['BUILDING-WIDE CONDITION'] != 1) & (df_311_cleaned['status'] == 'Closed')].index
print("There are {} records where the resolution description claims the complaint is open but the status claims it's closed. Dropping these if not a building-wide duplicate complaint (where the complaint status is sometimes for the original complaint).".format(len(list(resolution_open_status_not))))

#Dropping these
df_311_cleaned.drop(list(resolution_closed_status_not) + list(resolution_open_status_not), inplace = True)

#Comparing status to closed_date
date_closed_status_not = df_311_cleaned[(~df_311_cleaned['closed_date'].isna()) & (df_311_cleaned['BUILDING-WIDE CONDITION'] != 1) & (df_311_cleaned['status'] != 'Closed')].index
print("There are {} records where the closed_date claims the complaint is closed but the status claims it's open. Dropping these.".format(len(list(date_closed_status_not))))
date_open_status_not = df_311_cleaned[(df_311_cleaned['closed_date'].isna()) & (df_311_cleaned['BUILDING-WIDE CONDITION'] != 1) & (df_311_cleaned['status'] == 'Closed')].index
print("There are {} records where the closed_date claims the complaint is open but the status claims it's closed. Dropping these if not a building-wide duplicate complaint (where the complaint status is sometimes for the original complaint).".format(len(list(date_open_status_not))))

#Dropping these
df_311_cleaned.drop(list(date_closed_status_not) + list(date_open_status_not), inplace = True)

There are 29 records where the resolution description claims the complaint is closed but the status claims it's open. Dropping these.
There are 51921 records where the resolution description claims the complaint is open but the status claims it's closed. Dropping these if not a building-wide duplicate complaint (where the complaint status is sometimes for the original complaint).
There are 7038 records where the closed_date claims the complaint is closed but the status claims it's open. Dropping these.
There are 190 records where the closed_date claims the complaint is open but the status claims it's closed. Dropping these if not a building-wide duplicate complaint (where the complaint status is sometimes for the original complaint).


*I've contacted someone at the HPD department to confirm my understanding of these discrepancies. For right now I've used common sense to drop the ones that seem problematic, erring on the side of dropping when I was not sure whether an anomaly was intentional or an error.*

In [105]:
#If any records are missing a unique key or have duplicates, drop them
missing_key = df_311_cleaned['unique_key'].isna()
if any(missing_key):
    print('{} records are missing their unique identifying key. Dropping these'.format(sum(missing_key)))
    df_311_cleaned = df_311_cleaned[~missing_key]
    
print("Also dropping any duplicate keys")
df_311_cleaned.drop_duplicates('unique_key', inplace = True)

Also dropping any duplicate keys


## Section 6
### Analysis and tables for creating visualizations in Tableau

*Analyzes resolution_description text_tags and exports tables that can be loaded into Tableau to generate visualizations*

Much of this analysis could in theory be done in Tableau, but because this dataset is so large (~6 million rows) it exceeds the limits of Tableau Public. (As a JSON it exceeds Tableau's size limit, and as a .xlsx file, the number of rows exceeds the ~1 million limit of Excel. CSV's do not have a rows limit, but Tableau Public doesn't accept them as a file type.) By conducting the analysis in Python and exporting a small aggregated, analysis table for each visualization, I can get around this. 

##### Subset data to the fields I'm interested in analyzing

In [353]:
#Extract the fields of interest
non_tag_cols = ['unique_key','created_date', 'closed_date','incident_zip', 'latitude', 'longitude','borough', 'resolution_action_updated_date','complaint_type', 'cleaned_resolution_description', 'ultimate_outcome', 'ultimate_process']
columns_extract = non_tag_cols + all_tags
df_311_cleaned_wide = df_311_cleaned[columns_extract]

#Drop the records that don't have resolution_descriptions since this is the focus of my analysis
df_311_cleaned_wide = df_311_cleaned_wide[~df_311_cleaned_wide['cleaned_resolution_description'].isna()]
print("{} out of {} records have resolution_descriptions. Dropping the ones without.".format(df_311_cleaned_wide.shape[0], df_311_cleaned.shape[0]))

#Drop literature requests from dataset
print("Dropping literature reviews and focusing only on building issues")
literature_tags = [column for column in df_311_cleaned_wide.columns if 'LITERATURE' in column]
df_311_cleaned_wide.drop(columns = literature_tags, inplace = True)

print("The final size of the dataset is: {} complaint records".format(df_311_cleaned_wide.shape[0]))

6116870 out of 6124686 records have resolution_descriptions. Dropping the ones without.
Dropping literature reviews and focusing only on building issues
The final size of the dataset is: 6116870 complaint records


##### Add calculated columns that are helpful for all analyses


In [354]:
df_311_cleaned_wide['created_year'] = pd.DatetimeIndex(df_311_cleaned_wide['created_date']).year
df_311_cleaned_wide['minutes_to_close_claim'] = (df_311_cleaned_wide['closed_date'] - df_311_cleaned_wide['created_date']).astype('timedelta64[m]')
df_311_cleaned_wide['days_to_close_claim'] = df_311_cleaned_wide['minutes_to_close_claim']/(60*24)

##### Fix zip codes which got read as numeric and lost the leading 0s

In [355]:
df_311_cleaned_wide['incident_zip_str'] = df_311_cleaned_wide['incident_zip'].apply(lambda x: str(round(x)) if pd.notnull(x) else 'NaN')
df_311_cleaned_wide['incident_zip_str'] = df_311_cleaned_wide['incident_zip_str'].apply(lambda x: x.zfill(5) if x!='NaN' else 'NaN')

##### Many of the process, outcome, and complaint_type columns are redundant so grouping values into non-redundant categories

In [183]:
#Export full set of outcome and process text tags to manually group (can't use text_tag_dict.csv because it doesn't created the ones generated in cleaning of ultimate columns)
pd.DataFrame({'text_tag':df_311_cleaned['ultimate_outcome'].unique()}).to_csv('cleaning_resolution_text/all_ultimate_outcome_text_tags.csv')
pd.DataFrame({'text_tag':df_311_cleaned['ultimate_process'].unique()}).to_csv('cleaning_resolution_text/all_ultimate_process_text_tags.csv')
pd.DataFrame({'text_tag':df_311_cleaned['complaint_type'].unique()}).to_csv('cleaning_resolution_text/all_complaint_type_categories.csv')

In [356]:
#Load the manually created groups in
grouped_outcome = pd.read_csv('cleaning_resolution_text/grouped_ultimate_outcome_text_tags.csv')
grouped_process = pd.read_csv('cleaning_resolution_text/grouped_ultimate_process_text_tags.csv')
grouped_complaint_type = pd.read_csv('cleaning_resolution_text/grouped_complaint_type_categories.csv')

#Merge in grouped categories for process and outcome ultimate columns
df_311_cleaned_wide = df_311_cleaned_wide.merge(grouped_outcome, left_on = 'ultimate_outcome', right_on = 'text_tag', how = 'left')
df_311_cleaned_wide.drop(columns = 'text_tag', inplace = True)
df_311_cleaned_wide.rename(columns = {'grouped_text_tag':'grouped_ultimate_outcome'}, inplace = True)
display(df_311_cleaned_wide.head(1))
df_311_cleaned_wide = df_311_cleaned_wide.merge(grouped_process, left_on = 'ultimate_process', right_on = 'text_tag', how = 'left')
df_311_cleaned_wide.drop(columns = 'text_tag', inplace = True)
df_311_cleaned_wide.rename(columns = {'grouped_text_tag':'grouped_ultimate_process'}, inplace = True)
display(df_311_cleaned_wide.head(1))
df_311_cleaned_wide = df_311_cleaned_wide.merge(grouped_complaint_type, on ='complaint_type', how = 'left')
display(df_311_cleaned_wide.head(1))


,unique_key,created_date,closed_date,incident_zip,latitude,longitude,borough,resolution_action_updated_date,complaint_type,cleaned_resolution_description,ultimate_outcome,ultimate_process,BUILDING-WIDE CONDITION,HEAT OR HOT WATER,COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT,CONDITIONS WERE CORRECTED,COMPLAINT WAS ADDRESSED,COMPLAINT HAS BEEN CLOSED,NO VIOLATIONS WERE ISSUED,VIOLATIONS WERE PREVIOUSLY ISSUED,VIOLATION WAS ISSUED,IDENTIFIED POTENTIAL LEAD,ALSO IDENTIFIED POTENTIAL LEAD,CLOSED THIS COMPLAINT ADMINISTRATIVELY,SECTION 8 FAILURE WAS ISSUED,...,UNABLE TO ACCESS,BY INSPECTING ANOTHER APARTMENT,MAY ATTEMPT TO CONTACT YOU,WILL ATTEMPT TO CONTACT YOU,INSPECTION TO TEST THE PAINT FOR LEAD,CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION,IF THE CONDITION STILL EXISTS PLEASE FILE A NEW COMPLAINT,INFORMATION ABOUT SPECIFIC VIOLATIONS IS AVAILABLE AT WWW. NYC. GOV/HPD,IF THE CONDITIONS STILL EXIST AND AN INSPECTION IS REQUIRED PLEASE CONTACT THE BOROUGH OFFICE WITH YOUR COMPLAINT NUMBER,PLEASE NOTE YOUR SERVICE REQUEST NUMBER FOR FUTURE REFERENCE,INFORMATION ABOUT INSPECTION RESULTS CAN BE FOUND THROUGH HPD'S WEBSITE,MORE INFORMATION CAN BE FOUND THROUGH HPD'S WEBSITE,TENANT AND THE PROPERTY OWNER WILL RECEIVE A NOTICE IN THE MAIL,SCHEDULE A FOLLOW-UP INSPECTION,IF IT HAS NOT BEEN RECEIVED PLEASE REQUEST IT AGAIN,311 CAN SUBMIT A NEW REQUEST TO BE SENT BY DIRECT MAIL,IF IT IS NOT RECEIVED PLEASE CALL 311 AND SUBMIT A NEW REQUEST,MAY CONDUCT AN INSPECTION,VIOLATIONS WERE ISSUED,AT THE TIME OF THE INSPECTION,created_year,minutes_to_close_claim,days_to_close_claim,incident_zip_str,grouped_ultimate_outcome
0,47732947,2020-09-29 17:51:04,NaT,11235.0,40.587653,-73.957289,BROOKLYN,2020-09-29,WATER LEAK,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,COMPLAINT CONDITIONS ARE STILL OPEN,RESOLUTION PROCESS UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2020,NaN,NaN,11235,COMPLAINT STILL OPEN


,unique_key,created_date,closed_date,incident_zip,latitude,longitude,borough,resolution_action_updated_date,complaint_type,cleaned_resolution_description,ultimate_outcome,ultimate_process,BUILDING-WIDE CONDITION,HEAT OR HOT WATER,COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT,CONDITIONS WERE CORRECTED,COMPLAINT WAS ADDRESSED,COMPLAINT HAS BEEN CLOSED,NO VIOLATIONS WERE ISSUED,VIOLATIONS WERE PREVIOUSLY ISSUED,VIOLATION WAS ISSUED,IDENTIFIED POTENTIAL LEAD,ALSO IDENTIFIED POTENTIAL LEAD,CLOSED THIS COMPLAINT ADMINISTRATIVELY,SECTION 8 FAILURE WAS ISSUED,...,BY INSPECTING ANOTHER APARTMENT,MAY ATTEMPT TO CONTACT YOU,WILL ATTEMPT TO CONTACT YOU,INSPECTION TO TEST THE PAINT FOR LEAD,CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION,IF THE CONDITION STILL EXISTS PLEASE FILE A NEW COMPLAINT,INFORMATION ABOUT SPECIFIC VIOLATIONS IS AVAILABLE AT WWW. NYC. GOV/HPD,IF THE CONDITIONS STILL EXIST AND AN INSPECTION IS REQUIRED PLEASE CONTACT THE BOROUGH OFFICE WITH YOUR COMPLAINT NUMBER,PLEASE NOTE YOUR SERVICE REQUEST NUMBER FOR FUTURE REFERENCE,INFORMATION ABOUT INSPECTION RESULTS CAN BE FOUND THROUGH HPD'S WEBSITE,MORE INFORMATION CAN BE FOUND THROUGH HPD'S WEBSITE,TENANT AND THE PROPERTY OWNER WILL RECEIVE A NOTICE IN THE MAIL,SCHEDULE A FOLLOW-UP INSPECTION,IF IT HAS NOT BEEN RECEIVED PLEASE REQUEST IT AGAIN,311 CAN SUBMIT A NEW REQUEST TO BE SENT BY DIRECT MAIL,IF IT IS NOT RECEIVED PLEASE CALL 311 AND SUBMIT A NEW REQUEST,MAY CONDUCT AN INSPECTION,VIOLATIONS WERE ISSUED,AT THE TIME OF THE INSPECTION,created_year,minutes_to_close_claim,days_to_close_claim,incident_zip_str,grouped_ultimate_outcome,grouped_ultimate_process
0,47732947,2020-09-29 17:51:04,NaT,11235.0,40.587653,-73.957289,BROOKLYN,2020-09-29,WATER LEAK,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,COMPLAINT CONDITIONS ARE STILL OPEN,RESOLUTION PROCESS UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2020,NaN,NaN,11235,COMPLAINT STILL OPEN,RESOLUTION PROCESS UNKNOWN


,unique_key,created_date,closed_date,incident_zip,latitude,longitude,borough,resolution_action_updated_date,complaint_type,cleaned_resolution_description,ultimate_outcome,ultimate_process,BUILDING-WIDE CONDITION,HEAT OR HOT WATER,COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT,CONDITIONS WERE CORRECTED,COMPLAINT WAS ADDRESSED,COMPLAINT HAS BEEN CLOSED,NO VIOLATIONS WERE ISSUED,VIOLATIONS WERE PREVIOUSLY ISSUED,VIOLATION WAS ISSUED,IDENTIFIED POTENTIAL LEAD,ALSO IDENTIFIED POTENTIAL LEAD,CLOSED THIS COMPLAINT ADMINISTRATIVELY,SECTION 8 FAILURE WAS ISSUED,...,MAY ATTEMPT TO CONTACT YOU,WILL ATTEMPT TO CONTACT YOU,INSPECTION TO TEST THE PAINT FOR LEAD,CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION,IF THE CONDITION STILL EXISTS PLEASE FILE A NEW COMPLAINT,INFORMATION ABOUT SPECIFIC VIOLATIONS IS AVAILABLE AT WWW. NYC. GOV/HPD,IF THE CONDITIONS STILL EXIST AND AN INSPECTION IS REQUIRED PLEASE CONTACT THE BOROUGH OFFICE WITH YOUR COMPLAINT NUMBER,PLEASE NOTE YOUR SERVICE REQUEST NUMBER FOR FUTURE REFERENCE,INFORMATION ABOUT INSPECTION RESULTS CAN BE FOUND THROUGH HPD'S WEBSITE,MORE INFORMATION CAN BE FOUND THROUGH HPD'S WEBSITE,TENANT AND THE PROPERTY OWNER WILL RECEIVE A NOTICE IN THE MAIL,SCHEDULE A FOLLOW-UP INSPECTION,IF IT HAS NOT BEEN RECEIVED PLEASE REQUEST IT AGAIN,311 CAN SUBMIT A NEW REQUEST TO BE SENT BY DIRECT MAIL,IF IT IS NOT RECEIVED PLEASE CALL 311 AND SUBMIT A NEW REQUEST,MAY CONDUCT AN INSPECTION,VIOLATIONS WERE ISSUED,AT THE TIME OF THE INSPECTION,created_year,minutes_to_close_claim,days_to_close_claim,incident_zip_str,grouped_ultimate_outcome,grouped_ultimate_process,grouped_complaint_type
0,47732947,2020-09-29 17:51:04,NaT,11235.0,40.587653,-73.957289,BROOKLYN,2020-09-29,WATER LEAK,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,COMPLAINT CONDITIONS ARE STILL OPEN,RESOLUTION PROCESS UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2020,NaN,NaN,11235,COMPLAINT STILL OPEN,RESOLUTION PROCESS UNKNOWN,PLUMBING


### Analyses
The below generate the tables loaded into Tableau (some analyses are used for multiple figures and some are only used for one).

#### Analysis 1
*Do the outcomes of complaints vary by the process used to resolve them? Does this vary over time?*

In [250]:
#Drop the ones missing created_date
to_analyze = df_311_cleaned_wide[(~df_311_cleaned_wide['created_date'].isna())]

tot_by_year_process = pd.DataFrame(to_analyze.groupby(['created_year', 'grouped_ultimate_process']).agg({'unique_key':'nunique'})).reset_index()
tot_by_year_process.columns = ['created_year', 'process', 'tot_num_complaints_by_process']

tot_by_year_process_outcome = pd.DataFrame(to_analyze.groupby(['created_year', 'grouped_ultimate_process', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
tot_by_year_process_outcome.columns = ['created_year', 'process', 'outcome', 'num_complaints_of_outcome_X']

tot_by_year_process_merged = tot_by_year_process.merge(tot_by_year_process_outcome, on = ['created_year', 'process'], how = 'left')

#Calculate percentages
tot_by_year_process_merged['percent_complaints_of_outcome_X'] = tot_by_year_process_merged['num_complaints_of_outcome_X']/tot_by_year_process_merged['tot_num_complaints_by_process']*100
tot_by_year_process_merged.head(2)

,created_year,process,tot_num_complaints_by_process,outcome,num_complaints_of_outcome_X,percent_complaints_of_outcome_X
0,2010,ATTEMPTED AND FAILED INSPECTION,153534,COMPLAINT STILL OPEN,152259,99.169565
1,2010,ATTEMPTED AND FAILED INSPECTION,153534,VIOLATIONS WERE ISSUED,1275,0.830435


In [252]:
#Export for Tableau
tot_by_year_process_merged.to_excel('cleaned_data/outcome_by_process_with_years.xlsx')

#### Analysis 2
*How does the resolution process and outcome vary across complaint types? Has this changed over the last decade?*

Calculating percent of complaints in each category that employ each resolution process and have each outcome

In [254]:
#Drop the ones missing created_date and complaint type
to_analyze = df_311_cleaned_wide[(~df_311_cleaned_wide['created_date'].isna())&(~df_311_cleaned_wide['grouped_complaint_type'].isna())]

tot_by_year_complaint_type = pd.DataFrame(to_analyze.groupby(['created_year', 'grouped_complaint_type']).agg({'unique_key':'nunique'})).reset_index()
tot_by_year_complaint_type.columns = ['created_year', 'complaint_type', 'tot_num_complaints_by_complaint_type']

tot_by_year_complaint_type_process = pd.DataFrame(to_analyze.groupby(['created_year', 'grouped_complaint_type', 'grouped_ultimate_process'])['unique_key'].nunique()).reset_index()
tot_by_year_complaint_type_process.columns = ['created_year', 'complaint_type','process', 'num_complaints_of_process_X']

tot_by_year_complaint_type_outcome = pd.DataFrame(to_analyze.groupby(['created_year', 'grouped_complaint_type', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
tot_by_year_complaint_type_outcome.columns = ['created_year', 'complaint_type','outcome', 'num_complaints_of_outcome_X']

tot_by_year_complaint_type_merged = tot_by_year_complaint_type.merge(tot_by_year_complaint_type_process, on = ['created_year', 'complaint_type'], how = 'left')
tot_by_year_complaint_type_merged = tot_by_year_complaint_type_merged.merge(tot_by_year_complaint_type_outcome, on = ['created_year', 'complaint_type'], how = 'left')

#Calculate percents
tot_by_year_complaint_type_merged['percent_complaints_of_outcome_X'] = tot_by_year_complaint_type_merged['num_complaints_of_outcome_X']/tot_by_year_complaint_type_merged['tot_num_complaints_by_complaint_type']*100
tot_by_year_complaint_type_merged['percent_complaints_of_process_X'] = tot_by_year_complaint_type_merged['num_complaints_of_process_X']/tot_by_year_complaint_type_merged['tot_num_complaints_by_complaint_type']*100
display(tot_by_year_complaint_type_merged.head(2))

In [255]:
#Export to excel
tot_by_year_complaint_type_merged.to_excel("cleaned_data/process_and_outcome_by_complaint_type.xlsx")

#### Analysis 3
*Does the outcome and resolution process of complaints vary across neighborhood? Has this varied in the last decade?*

- Calculating percent of complaints in zipcode that result in each outcome and employ each resolution process by year
- Also calculating other interesting stats at the year/zip level: average resolution time, percent complaints that are for building-wide complaints

In [357]:
#Drop records missing created date and zip
to_analyze = df_311_cleaned_wide[(~df_311_cleaned_wide['created_date'].isna()) & (df_311_cleaned_wide['incident_zip_str'] !='NaN')]

In [358]:
'''Use reverse geocodoing google API to get borough from zipcode where borough is missing in 311 dataset to_analyze
'''

#Zipcodes that are missing borough in 311 dataset
zipcodes_to_geocode = list(to_analyze[to_analyze['borough'] == 'Unspecified']['incident_zip_str'].unique())

#Use geocoding API
api_key = ''#INSERT API KEY HERE
zipcodes_df = pd.DataFrame(columns = ['incident_zip_str', 'borough'])
failed_zips = []
for zipcode in zipcodes_to_geocode:
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={},US&key={}'.format(zipcode, api_key)
    payload = {}
    headers= {}
    response = requests.request("GET", url, headers=headers, data = payload)
    if (response.status_code == 200):
        address = pd.DataFrame(json.loads(response.text)['results'][0]['address_components'])
        borough_name_list = address[address['types'].map(set(['sublocality_level_1']).issubset)]['short_name'].values
        if len(borough_name_list) > 0:
            borough_name = address[address['types'].map(set(['sublocality_level_1']).issubset)]['short_name'].values[0]
            row_append = {'incident_zip_str':zipcode, 'borough':borough_name}
            zipcodes_df = zipcodes_df.append(row_append, ignore_index = True)
        else:
            failed_zips.append(zipcode)
    else:
        print('error with geocoding')
        print(response.text)
        print(zipcode)

#Manually look up boroughs of zipcodes that Google Geocoding API don't return borough of
print('The following zipcodes need to be manually looked up {}'.format(failed_zips))
rows_append = [{'incident_zip_str':'10009', 'borough':'Manhattan'}, {'incident_zip_str':'11694', 'borough':'Queens'}, {'incident_zip_str':'11101', 'borough':'Queens'}, {'incident_zip_str':'11102', 'borough':'Queens'}, {'incident_zip_str':'11422', 'borough':'Queens'},{'incident_zip_str':'11429', 'borough':'Queens'},{'incident_zip_str':'11693', 'borough':'Queens'},{'incident_zip_str':'10038', 'borough':'Manhattan'},{'incident_zip_str':'11414', 'borough':'Queens'},{'incident_zip_str':'11363', 'borough':'Queens'},{'incident_zip_str':'11001', 'borough':'Queens'},{'incident_zip_str':'11040', 'borough':'Queens'}]
for row in rows_append:
    zipcodes_df = zipcodes_df.append(row, ignore_index = True)
    
#Make borough spelling/formatting returned by API match df_311
zipcodes_df['borough'] = zipcodes_df['borough'].str.upper()
zipcodes_df['borough'] = zipcodes_df['borough'].apply(lambda row: row.strip('THE '))
zipcodes_df['borough'].unique()

#Merge in missing boroughs
zipcodes_df.rename(columns = {'borough':'borough_API'}, inplace = True)
to_analyze = to_analyze.merge(zipcodes_df, on = 'incident_zip_str', how = 'left')

#If missing borough use API borough
to_analyze['borough'] = to_analyze.apply(lambda row: row['borough'] if row['borough']!='Unspecified' else row['borough_API'], axis = 1)
to_analyze.head(2)

#Correct two boroughs that are inconsistently labeled
to_analyze.loc[to_analyze['incident_zip_str']=='10463', 'borough'] = 'BRONX'
to_analyze.loc[to_analyze['incident_zip_str']=='11416', 'borough'] = 'QUEENS'


The following zipcodes need to be manually looked up ['10009', '11694', '11101', '11102', '11422', '11429', '11693', '10038', '11414', '11363', '11001', '11040']


,unique_key,created_date,closed_date,incident_zip,latitude,longitude,borough,resolution_action_updated_date,complaint_type,cleaned_resolution_description,ultimate_outcome,ultimate_process,BUILDING-WIDE CONDITION,HEAT OR HOT WATER,COMPLAINT STATUS IS FOR THE INITIAL COMPLAINT,CONDITIONS WERE CORRECTED,COMPLAINT WAS ADDRESSED,COMPLAINT HAS BEEN CLOSED,NO VIOLATIONS WERE ISSUED,VIOLATIONS WERE PREVIOUSLY ISSUED,VIOLATION WAS ISSUED,IDENTIFIED POTENTIAL LEAD,ALSO IDENTIFIED POTENTIAL LEAD,CLOSED THIS COMPLAINT ADMINISTRATIVELY,SECTION 8 FAILURE WAS ISSUED,...,WILL ATTEMPT TO CONTACT YOU,INSPECTION TO TEST THE PAINT FOR LEAD,CONDUCTED OR ATTEMPTED TO CONDUCT AN INSPECTION,IF THE CONDITION STILL EXISTS PLEASE FILE A NEW COMPLAINT,INFORMATION ABOUT SPECIFIC VIOLATIONS IS AVAILABLE AT WWW. NYC. GOV/HPD,IF THE CONDITIONS STILL EXIST AND AN INSPECTION IS REQUIRED PLEASE CONTACT THE BOROUGH OFFICE WITH YOUR COMPLAINT NUMBER,PLEASE NOTE YOUR SERVICE REQUEST NUMBER FOR FUTURE REFERENCE,INFORMATION ABOUT INSPECTION RESULTS CAN BE FOUND THROUGH HPD'S WEBSITE,MORE INFORMATION CAN BE FOUND THROUGH HPD'S WEBSITE,TENANT AND THE PROPERTY OWNER WILL RECEIVE A NOTICE IN THE MAIL,SCHEDULE A FOLLOW-UP INSPECTION,IF IT HAS NOT BEEN RECEIVED PLEASE REQUEST IT AGAIN,311 CAN SUBMIT A NEW REQUEST TO BE SENT BY DIRECT MAIL,IF IT IS NOT RECEIVED PLEASE CALL 311 AND SUBMIT A NEW REQUEST,MAY CONDUCT AN INSPECTION,VIOLATIONS WERE ISSUED,AT THE TIME OF THE INSPECTION,created_year,minutes_to_close_claim,days_to_close_claim,incident_zip_str,grouped_ultimate_outcome,grouped_ultimate_process,grouped_complaint_type,borough_API
0,47732947,2020-09-29 17:51:04,NaT,11235.0,40.587653,-73.957289,BROOKLYN,2020-09-29,WATER LEAK,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,COMPLAINT CONDITIONS ARE STILL OPEN,RESOLUTION PROCESS UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2020,NaN,NaN,11235,COMPLAINT STILL OPEN,RESOLUTION PROCESS UNKNOWN,PLUMBING,BROOKLYN
1,47732845,2020-09-29 00:22:57,NaT,11225.0,40.663922,-73.950589,BROOKLYN,2020-09-29,UNSANITARY CONDITION,The following complaint conditions are still open. HPD may attempt to contact you to verify the correction of the condition or may conduct an inspection.,COMPLAINT CONDITIONS ARE STILL OPEN,RESOLUTION PROCESS UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2020,NaN,NaN,11225,COMPLAINT STILL OPEN,RESOLUTION PROCESS UNKNOWN,UNSANITARY CONDITION/SAFETY,BROOKLYN


In [363]:
#Generate analysis by year and zip including boroughs as column
tot_by_year_zipcode = pd.DataFrame(to_analyze.groupby(['created_year', 'incident_zip_str']).agg({'unique_key':'nunique','days_to_close_claim':'mean', 'BUILDING-WIDE CONDITION':'sum', 'borough':'first'})).reset_index()
tot_by_year_zipcode.columns = ['created_year', 'incident_zip', 'tot_num_complaints', 'avg_resolution_time_days', 'num_complaints_building_wide_condition', 'borough']

tot_by_year_zipcode_outcome = pd.DataFrame(to_analyze.groupby(['created_year', 'incident_zip_str', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
tot_by_year_zipcode_outcome.columns = ['created_year', 'incident_zip', 'outcome', 'num_complaints_of_outcome_X']
if (tot_by_year_zipcode_outcome['num_complaints_of_outcome_X'].sum() != tot_by_year_zipcode['tot_num_complaints'].sum()):
    print("Error. Percents won't sum to 100 in graph. There are records missing ultimate outcomes.")

tot_by_year_zipcode_process = pd.DataFrame(to_analyze.groupby(['created_year', 'incident_zip_str', 'grouped_ultimate_process'])['unique_key'].nunique()).reset_index()
tot_by_year_zipcode_process.columns = ['created_year', 'incident_zip', 'process', 'num_complaints_of_process_X']
if (tot_by_year_zipcode_process['num_complaints_of_process_X'].sum() != tot_by_year_zipcode['tot_num_complaints'].sum()):
    print("Error. Percents won't sum to 100 in graph. There are records missing process outcomes.")

tot_by_year_zipcode_merged = tot_by_year_zipcode.merge(tot_by_year_zipcode_outcome, on = ['created_year', 'incident_zip'], how = 'left')
# display(tot_by_year_zipcode_merged.head(5))
tot_by_year_zipcode_merged = tot_by_year_zipcode_merged.merge(tot_by_year_zipcode_process, on = ['created_year', 'incident_zip'], how = 'left')
# display(tot_by_year_zipcode_merged.head(5))

#Calculate percents
tot_by_year_zipcode_merged['percent_complaints_of_outcome_X'] = tot_by_year_zipcode_merged['num_complaints_of_outcome_X']/tot_by_year_zipcode_merged['tot_num_complaints']*100
tot_by_year_zipcode_merged['percent_complaints_of_process_X'] = tot_by_year_zipcode_merged['num_complaints_of_process_X']/tot_by_year_zipcode_merged['tot_num_complaints']*100
tot_by_year_zipcode_merged['percent_complaints_building_wide'] = tot_by_year_zipcode_merged['num_complaints_building_wide_condition']/tot_by_year_zipcode_merged['tot_num_complaints']*100

In [364]:
tot_by_year_zipcode_merged.head(2)

,created_year,incident_zip,tot_num_complaints,avg_resolution_time_days,num_complaints_building_wide_condition,borough,outcome,num_complaints_of_outcome_X,process,num_complaints_of_process_X,percent_complaints_of_outcome_X,percent_complaints_of_process_X,percent_complaints_building_wide
0,2010,10001,1197,12.080335,363,MANHATTAN,COMPLAINT STILL OPEN,352,ATTEMPTED AND FAILED INSPECTION,352,29.40685,29.406850,30.325815
1,2010,10001,1197,12.080335,363,MANHATTAN,COMPLAINT STILL OPEN,352,CONTACTED TENANT,479,29.40685,40.016708,30.325815


In [365]:
#Write table to excel for Tableau
tot_by_year_zipcode_merged.to_excel("cleaned_data/process_outcome_by_zipcode.xlsx")

In [261]:
#Write initial table to excel for Tableau
tot_by_year_zipcode.to_excel("cleaned_data/tot_number_complaints_by_zip.xlsx")

#### Bubble chart of resolution text tokenization
*How many complaints does each text_tag correspond to in each year? (ie. how has the process/outcomes of HPD changed over time?)*

In [273]:
#Reshape from wide to long
text_tags_in_cleaned = list(set(text_tag_dict['text_tag'].unique()).intersection(set(df_311_cleaned_wide.columns)))
to_analyze = df_311_cleaned_wide[['unique_key', 'created_year'] + text_tags_in_cleaned]
text_tag_view = to_analyze.melt(id_vars = ['unique_key', 'created_year'], var_name = 'text_tag', value_name = 'text_tag_bool')

#Drop all the redundant 0's that we don't need
text_tag_view = text_tag_view[text_tag_view['text_tag_bool']!=0]

#Merge in tag_types
text_tag_view = text_tag_view.merge(text_tag_dict, on = 'text_tag', how ='left')

#Do text tag analysis over time - add in tags to the tags to categorize them as good/bad or neutral/not?
text_tag_analyzed = text_tag_view.groupby(['text_tag', 'created_year']).agg({'tag_type':'first','unique_key':'count'}).reset_index()


In [274]:
#Write to file for Tableau
text_tag_analyzed.to_excel('cleaned_data/text_tag_analyzed.xlsx')